## E7. 프로젝트 - Movielens 영화 추천 실습

### 1) 데이터 준비와 전처리

In [1]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [4]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre', 'count'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre,count
0,1,Toy Story (1995),Animation|Children's|Comedy,NaN
1,2,Jumanji (1995),Adventure|Children's|Fantasy,NaN
2,3,Grumpier Old Men (1995),Comedy|Romance,NaN
3,4,Waiting to Exhale (1995),Comedy|Drama,NaN
4,5,Father of the Bride Part II (1995),Comedy,NaN


### 2) 분석해 봅시다.

In [5]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [6]:
# ratings에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [7]:
# 가장 인기있는 영화 30개(인기순)
top_movies = pd.merge(ratings, movies, how='left', on='movie_id')
#top_movies.head()

tops = top_movies.sort_values(ascending=False, by=['count_x']).head(30)
tops['title']

0                    One Flew Over the Cuckoo's Nest (1975)
327966                               Midnight Cowboy (1969)
651588    Star Wars: Episode VI - Return of the Jedi (1983)
327977    Horseman on the Roof, The (Hussard sur le toit...
327976                      Who Framed Roger Rabbit? (1988)
651585                                     True Lies (1994)
176729                                          Rudy (1993)
176730                              Schindler's List (1993)
327975                                Bank Dick, The (1940)
327974                                  Bikini Beach (1964)
327973                                   Beach Party (1963)
176734                          Fish Called Wanda, A (1988)
176735                                  Best in Show (2000)
176736                     Phantom of the Opera, The (1943)
176737                                     Duck Soup (1933)
176738                          Being John Malkovich (1999)
327972                                 D

### 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

In [8]:
#내가 선호하는 영화 5가지 평점 추가
#1909::X-Files: Fight the Future, The (1998)::Mystery|Sci-Fi|Thriller
#3798::What Lies Beneath (2000)::Thriller
#2012::Back to the Future Part III (1990)::Comedy|Sci-Fi|Western
#1::Toy Story (1995)::Animation|Children's|Comedy
#595::Beauty and the Beast (1991)::Animation|Children's|Musical

my_fav_movie_ids = [1909,3798,2012,1,595]

# 기존 user_id 중 가장 큰 값이 6040이므로 6041로 추가
my_movie_list = pd.DataFrame({'user_id':[6041]*5, 'movie_id':my_fav_movie_ids, 'count':5,'timestamp':0})
ratings = ratings.append(my_movie_list)
ratings = ratings.reset_index(drop=True)

In [9]:
# 잘 추가되었는지 확인
ratings.sort_values(ascending=False, by=['user_id']).head(10)

,user_id,movie_id,count,timestamp
836482,6041,595,5,0
836481,6041,1,5,0
836480,6041,2012,5,0
836479,6041,3798,5,0
836478,6041,1909,5,0
836296,6040,318,4,956704257
836290,6040,2575,4,997453762
836291,6040,306,4,957716754
836292,6040,3521,5,956715569
836293,6040,1921,4,997454464


In [10]:
# index 만들기
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

user_to_idx = {v:k+1 for k,v in enumerate(user_unique)}
movie_to_idx = {v:k+1 for k,v in enumerate(movie_unique)}

### 4) CSR matrix를 직접 만들어 봅시다.

In [11]:
from scipy.sparse import csr_matrix

num_user =  ratings['user_id'].max()+1 #nunique()
num_movie = ratings['movie_id'].max()+1 #.nunique()

ratings.rename(columns={'count':'play'}, inplace=True)
csr_data = csr_matrix((ratings.play, (ratings.user_id, ratings.movie_id)),
                      shape = (num_user, num_movie))
csr_data

<6042x3953 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

### 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [12]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [13]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [14]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3953x6042 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [15]:
# 모델 훈련
als_model.fit(csr_data_transpose)

### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [16]:
# 5개 영화중 하나인 X-files 영화 (movie_id: 1909)와 나 (user_id: 6041)
seungho, xfiles = user_to_idx[6041], movie_to_idx[1909]
seungho_vector, xfiles_vector = als_model.user_factors[seungho], als_model.item_factors[xfiles]


In [17]:
np.dot(seungho_vector, xfiles_vector)

0.1769762

In [18]:
# 내가 선택하지 않은 다른 영화 (movie_id: 318)
othermovie = movie_to_idx[318]
othermovie_vector = als_model.item_factors[othermovie]

np.dot(seungho_vector, othermovie_vector)

-0.16234162

- 선호하는 영화는 양수, 선택하지 않은 영화는 음수로 차이가 납니다.

### 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [19]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}

def get_similar_movies(movie_id:int):
    similar_movie = als_model.similar_items(movie_id)
    return similar_movie

idx_to_movie_name = {movies.loc[i,'movie_id']:movies.loc[i,'title'] for i in range(len(movies))}

[[idx_to_movie_name.get(idx[0]),idx[1]] for idx in get_similar_movies(1909)]

[['X-Files: Fight the Future, The (1998)', 0.15649186],
 ['Contact (1997)', 0.09872331],
 ['Twelve Monkeys (1995)', 0.08513159],
 ['Pi (1998)', 0.08062054],
 ['Dark City (1998)', 0.08025623],
 ['Fifth Element, The (1997)', 0.07702747],
 ['Gattaca (1997)', 0.07647236],
 ['Mimic (1997)', 0.07550485],
 ['Dead Again (1991)', 0.07528206],
 ['Sneakers (1992)', 0.073200144]]

- 입력한 영화 X-Files는 0.16인데, 다른 영화들도 비슷하게 0.73 ~ 0.98 값을 보입니다.

### 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [20]:
user = user_to_idx[6041]
# recommend에서는 user*item CSR Matrix를 받습니다.
movies_recommended = als_model.recommend(user, csr_data, N=10, filter_already_liked_items=True)
movies_recommended

[[idx_to_movie_name.get(idx[0]), idx[0]] for idx in movies_recommended]

[['Right Stuff, The (1983)', 1231],
 ['Do the Right Thing (1989)', 3424],
 ['Heavenly Creatures (1994)', 247],
 ['Sex, Lies, and Videotape (1989)', 1186],
 ['Gandhi (1982)', 1293],
 ['Drugstore Cowboy (1989)', 3019],
 ["Boys Don't Cry (1999)", 2908],
 ['Insider, The (1999)', 3006],
 ['Duck Soup (1933)', 1256],
 ['To Kill a Mockingbird (1962)', 1207]]

In [21]:
#추천한 영화의 기여도
explain = als_model.explain(user, csr_data, itemid=1231)
[(idx_to_movie_name.get(i[0]),i[1]) for i in explain[1]]

[('My Left Foot (1989)', 0.06378766979696371),
 ('12 Angry Men (1957)', 0.05933977394731103),
 ('Raging Bull (1980)', 0.05756564831664069),
 ('Amadeus (1984)', 0.051070990292511664),
 ('Rain Man (1988)', 0.04911598979971528),
 ('This Is Spinal Tap (1984)', 0.048254011601186995),
 ('Verdict, The (1982)', 0.046556195564135046),
 ('Breaking Away (1979)', 0.042754097642694514),
 ('Killing Fields, The (1984)', 0.0409522940325645),
 ('Year of Living Dangerously (1982)', 0.039498935946695464)]

- 추천한 영화를 explain()에 넣었을 때, 기여한 영화 목록을 확인할 수 있었습니다.

- 유저 선호도 및, 아이템간 유사도, 기여도가 의미있게 측정되었습니다.